# Error Sensitivity

In [1]:
import sys
sys.path.insert(0, '../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm
from qiskit.providers.aer import AerSimulator
from qiskit.providers.fake_provider import FakeCasablanca

from loss_functions import *
from optimization import *
from quantum_maps import *
from quantum_tools import *
from experiments import *




#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

## Error in SPAM estimation

In [2]:
backend = AerSimulator()

n = 1
d = 2**n

input_list = [prepare_input(numberToBase(i, 6, n), return_mode = "unitary") for i in range(6**n)]
inputs = tf.convert_to_tensor(input_list, dtype = precision)

circuit_list = [prepare_input(numberToBase(i, 6, n), return_mode = "circuit_measure") for i in range(6**n)]

job = qk.execute(circuit_list, backend, shots = 20000, optimization_level = 0, seed_transpiler=42)

result = job.result()
counts_list = [result.get_counts(circuit) for circuit in circuit_list]
targets = counts_to_probs(counts_list)

In [ ]:
spam_model = SPAM(d=d,
                  #init = init_ideal(n),
                  #povm = povm_ideal(n),
                  #use_corr_mat = True,
                  optimizer = tf.optimizers.Adam(learning_rate=0.01))

spam_model.train(inputs = inputs,
                 targets = targets,
                 #inputs_val = input_map_list,
                 #targets_val = targets_map,
                 num_iter = 1000,
                 N = 6
                )

print(spam_model.init)
print(spam_model.povm)

  0%|          | 0/1000 [00:00<?, ?it/s]

0 0.17015073313392443
1 0.15565427032430756
2 0.14209720327379116
3 0.129918282910797
4 0.11833906368416774
5 0.10772500216245476
6 0.09808574177019258
7 0.089007840667939
8 0.08070364691411985
9 0.07317736913467776
10 0.06639523428443689
11 0.06031585523714031
12 0.05489226092346709
13 0.050073829970913405
14 0.04580806896014265
15 0.0420421838966159
16 0.0387243945225646
17 0.03580495661733213
18 0.0332368892526631
19 0.030976438530502082
20 0.028983331487773663
21 0.02722087825613305
22 0.025655971720360313
23 0.02425901971463375
24 0.023003831330093927
25 0.02186746910923424
26 0.02083007338404124
27 0.019874663247766496
28 0.018986919639531086
29 0.01815495842943557
30 0.017369103430870225
31 0.016621668844529522
32 0.01590675625825217
33 0.015220063626912143
34 0.014558696232337541
35 0.013920967304988022
36 0.013306180999001329
37 0.012714400056024375
38 0.012146209241135417
39 0.011602489476949768
40 0.01108421624846249
41 0.010592291533770152
42 0.010127413687689064
43 0.00968

328 0.00016109826833207743
329 0.00015981749552162488
330 0.0001585503255233904
331 0.00015729663667412284
332 0.00015605630747288744
333 0.00015482921484059707
334 0.0001536152326198536
335 0.0001524142303536494
336 0.0001512260723672966
337 0.00015005061716485014
338 0.00014888771713998065
339 0.00014773721859006404
340 0.0001465989620139278
341 0.0001454727826660002
342 0.00014435851133439895
343 0.0001432559753061128
344 0.0001421649994788049
345 0.00014108540757774222
346 0.0001400170234334246
347 0.0001389596722766622
348 0.00013791318200709647
349 0.00013687738439277985
350 0.00013585211616131006
351 0.00013483721994600302
352 0.00013383254505626812
353 0.00013283794804692955
354 0.0001318532930696502
355 0.000130878451997221
356 0.0001299133043209559
357 0.0001289577368304948
358 0.0001280116430939741
359 0.0001270749227648793
360 0.00012614748074794144
361 0.00012522922626184073
362 0.00012432007183913688
363 0.0001234199323050879
364 0.00012252872377532895
365 0.0001216463627

649 2.8123795391098232e-05
650 2.8019197658003047e-05
651 2.7915169642759156e-05
652 2.781170713302531e-05
653 2.7708805922535948e-05
654 2.760646181225472e-05
655 2.750467061337571e-05
656 2.7403428151977293e-05
657 2.7302730275060926e-05
658 2.7202572857661028e-05
659 2.7102951810579262e-05
660 2.700386308825632e-05
661 2.6905302696468586e-05
662 2.680726669910462e-05
663 2.670975122380148e-05
664 2.6612752466039848e-05
665 2.6516266691356575e-05
666 2.642029023548663e-05
667 2.6324819502513132e-05
668 2.6229850961022377e-05
669 2.6135381138320082e-05
670 2.6041406613322246e-05
671 2.5947924008141394e-05
672 2.5854929979160968e-05
673 2.5762421207775833e-05
674 2.5670394391617966e-05
675 2.5578846236396188e-05
676 2.5487773449159044e-05
677 2.5397172733071425e-05
678 2.5307040784163817e-05
679 2.521737429008371e-05
680 2.512816993111265e-05
681 2.5039424383214028e-05
682 2.4951134323022186e-05
683 2.486329643453867e-05
684 2.4775907417002143e-05
685 2.4688963993680635e-05
686 2.46024

In [40]:
backend = AerSimulator()

n = 2
d = 2**n

input_list = [prepare_input(numberToBase(i, 6, n), return_mode = "unitary") for i in range(6**n)]
inputs = tf.convert_to_tensor(input_list, dtype = precision)

circuit_list = [prepare_input(numberToBase(i, 6, n), return_mode = "circuit_measure") for i in range(6**n)]

job = qk.execute(circuit_list, backend, shots = 20000, optimization_level = 0, seed_transpiler=42)

result = job.result()
counts_list = [result.get_counts(circuit) for circuit in circuit_list]
targets = counts_to_probs(counts_list)

In [41]:
spam_model = SPAM(d=d,
                  #init = init_ideal(n),
                  #povm = povm_ideal(n),
                  #use_corr_mat = True,
                  optimizer = tf.optimizers.Adam(learning_rate=0.01))

spam_model.train(inputs = inputs,
                 targets = targets,
                 #inputs_val = input_map_list,
                 #targets_val = targets_map,
                 num_iter = 1000,
                 N = 36
                )

print(spam_model.init)
print(spam_model.povm)

  0%|          | 0/1000 [00:00<?, ?it/s]

0 0.8327440778587009
1 0.8225019057666594
2 0.8123111289377497
3 0.8021735596542908
4 0.7920680613567304
5 0.782013991741746
6 0.7720360103485825
7 0.7620454786146592
8 0.752146425892835
9 0.7423415403474267
10 0.7326223164491705
11 0.7229323494946382
12 0.713329977864575
13 0.7038200680662495
14 0.694290707552348
15 0.6848507129124657
16 0.6755063101196394
17 0.6662616635858912
18 0.6571186737886946
19 0.6480772769221909
20 0.639136305929234
21 0.6302419451378368
22 0.621430855231492
23 0.6127060743930051
24 0.6039564103770392
25 0.5952649908900671
26 0.5865374789172376
27 0.5778515955311008
28 0.5692028359744581
29 0.5604662755947862
30 0.5515485508118237
31 0.5425838270248171
32 0.5335581645900238
33 0.5244486208261789
34 0.5152740701599444
35 0.5060697721389241
36 0.49683992934094273
37 0.48738776566951636
38 0.4778872159838947
39 0.46835126336590055
40 0.4587540820012009
41 0.4490900847934507
42 0.43936421592038644
43 0.4296359807206729
44 0.4200255508149426
45 0.4106010544346161


344 0.0014137138323235864
345 0.001401269240023044
346 0.0013889821022533386
347 0.0013768505824461963
348 0.0013648728925029715
349 0.0013530472816506468
350 0.0013413720259654901
351 0.0013298454186572517
352 0.001318465761198982
353 0.0013072313553573948
354 0.0012961404961657852
355 0.0012851914658621104
356 0.001274382528799004
357 0.0012637119273145102
358 0.001253177878543594
359 0.001242778572136981
360 0.0012325121688435783
361 0.00122237679991058
362 0.001212370567245615
363 0.001202491544285193
364 0.0011927377775117686
365 0.0011831072885603185
366 0.0011735980768586558
367 0.0011642081227438475
368 0.001154935391002837
369 0.00114577783478304
370 0.0011367333998230566
371 0.001127800028955665
372 0.0011189756668325676
373 0.0011102582648249435
374 0.0011016457860490428
375 0.0010931362104700949
376 0.0010847275400305894
377 0.001076417803753501
378 0.001068205062765326
379 0.0010600874151821256
380 0.0010520630008047317
381 0.0010441300055631054
382 0.0010362866656551618
3

663 0.00026754756113397005
664 0.00026675724340888263
665 0.0002659716221447498
666 0.00026519067005085914
667 0.000264414360607591
668 0.00026364266790298175
669 0.0002628755664678692
670 0.00026211303111331935
671 0.00026135503677211943
672 0.00026060155834704265
673 0.0002598525705679116
674 0.0002591080478600228
675 0.0002583679642244378
676 0.00025763229313344323
677 0.0002569010074405681
678 0.00025617407930754476
679 0.00025545148014810836
680 0.0002547331805897202
681 0.0002540191504524443
682 0.00025330935874591596
683 0.00025260377368357224
684 0.0002519023627130849
685 0.00025120509256332024
686 0.0002505119293058396
687 0.00024982283842977924
688 0.00024913778492944384
689 0.00024845673340211506
690 0.0002477796481549902
691 0.0002471064933192721
692 0.0002464372329690849
693 0.0002457718312440244
694 0.0002451102524727354
695 0.0002444524612952721
696 0.000243798422782987
697 0.00024314810255357002
698 0.00024250146687875126
699 0.00024185848278468578
700 0.000241219118140

973 0.0001471388405492798
974 0.000146964351738314
975 0.00014679055275239593
976 0.00014661743941891798
977 0.00014644500770329762
978 0.00014627325371925813
979 0.00014610217373633145
980 0.0001459317641843457
981 0.00014576202165500688
982 0.00014559294290066908
983 0.00014542452482952913
984 0.00014525676449887888
985 0.00014508965910477328
986 0.00014492320596995096
987 0.0001447574025292491
988 0.0001445922463132477
989 0.00014442773493062876
990 0.0001442638660491742
991 0.00014410063737640275
992 0.0001439380466396029
993 0.0001437760915665456
994 0.0001436147698662506
995 0.0001434540792109544
996 0.00014329401721963606
997 0.0001431345814427384
998 0.0001429757693492051
999 0.00014281757831589636
tf.Tensor(
[[ 1.8622e-03+0.j     -2.4754e-04+0.0001j  3.6147e-03-0.0291j
   5.6011e-04+0.0011j]
 [-2.4754e-04-0.0001j  6.8374e-05+0.j     -1.2839e-03+0.0004j
   1.0706e-04-0.0001j]
 [ 3.6147e-03+0.0291j -1.2839e-03-0.0004j  9.9602e-01+0.j
  -2.1733e-02+0.029j ]
 [ 5.6011e-04-0.0011j 

In [15]:
backend = FakeCasablanca()

n = 3
d = 2**n

input_list = [prepare_input(numberToBase(i, 6, n), return_mode = "unitary") for i in range(6**n)]
inputs = tf.convert_to_tensor(input_list, dtype = precision)

circuit_list = [prepare_input(numberToBase(i, 6, n), return_mode = "circuit_measure") for i in range(6**n)]

job = qk.execute(circuit_list, backend, shots = 20000, optimization_level = 0, seed_transpiler=42)

result = job.result()
counts_list = [result.get_counts(circuit) for circuit in circuit_list]
targets = counts_to_probs(counts_list)

In [22]:
spam_model = SPAM(d=d,
                  #init = init_ideal(n),
                  #povm = povm_ideal(n),
                  use_corr_mat = True,
                  optimizer = tf.optimizers.Adam(learning_rate=0.01))

spam_model.pretrain(num_iter = 100,
                    targets = [init_ideal(n), povm_ideal(n)]
                )

  0%|          | 0/100 [00:00<?, ?it/s]

0 0.029952481496459474
1 0.02966490355343151
2 0.029382255230460973
3 0.029104500037654423
4 0.028831135774268983
5 0.028561683131386573
6 0.028295943718358946
7 0.028033507004217
8 0.02777385169648025
9 0.02751667318730765
10 0.02726191173797394
11 0.02700930018872496
12 0.02675871473328233
13 0.02651032406788742
14 0.02626331711453691
15 0.026017347841896486
16 0.025772179603906212
17 0.025527382151193323
18 0.025282200793842174
19 0.025037246125490718
20 0.0247928883761336
21 0.024548552482248534
22 0.024303914693939013
23 0.024058462697055978
24 0.02381222690638935
25 0.023565586652140123
26 0.023318584396176536
27 0.023070662322469807
28 0.02282178482662707
29 0.022571966927668774
30 0.022321036556429863
31 0.022069287174302885
32 0.02181748619499555
33 0.021564780896246073
34 0.021311215738020896
35 0.021057095812499066
36 0.02080252412182817
37 0.02054763470355863
38 0.020294592236696083
39 0.020040979933726322
40 0.019786904917263502
41 0.019532526914369193
42 0.019278595044167

In [23]:
spam_model.train(inputs = inputs,
                 targets = targets,
                 #inputs_val = input_map_list,
                 #targets_val = targets_map,
                 num_iter = 1000,
                 N = 216
                )

  0%|          | 0/1000 [00:00<?, ?it/s]

0 0.8027866949798034
1 0.7887139826649188
2 0.7742887320626051
3 0.7595057659848473
4 0.7463800145370939
5 0.7345573661428331
6 0.7224965770672618
7 0.710052940823482
8 0.6979361257076593
9 0.6867866816115709
10 0.6754168147435748
11 0.6640116069313767
12 0.6527809393986712
13 0.641753194292956
14 0.6307418496886515
15 0.6202794096719728
16 0.6102868625003481
17 0.6001656866792385
18 0.5898792021776925
19 0.5793377013322899
20 0.568916662878377
21 0.5592468800652829
22 0.5495852793771813
23 0.5396602126810026
24 0.5298618981671976
25 0.5200169216453093
26 0.5103589016568165
27 0.5010965257853494
28 0.49199259002522866
29 0.4827396015318261
30 0.47335008055323724
31 0.46393490199693893
32 0.45455768885929326
33 0.44559545860703065
34 0.43692548670333486
35 0.42810664254451253
36 0.41964282524418683
37 0.411227454516545
38 0.4026453068842481
39 0.3944994431890208
40 0.38646109094019676
41 0.3785422886357556
42 0.37048601412679283
43 0.36213040884100395
44 0.3541100444374913
45 0.34636564

341 0.0004401584996841711
342 0.00043848154019093593
343 0.0004369154972082074
344 0.0004361601585621104
345 0.00043595692136904864
346 0.00043587439911973747
347 0.000435764678848822
348 0.00043523004340211875
349 0.00043405329971973444
350 0.00043266603655472724
351 0.00043237175531304174
352 0.0004331088530647243
353 0.0004339653185632775
354 0.00043441922597415336
355 0.0004344971414903378
356 0.00043401501601632786
357 0.0004328656012580625
358 0.0004327494138897146
359 0.00043411390086846275
360 0.00043595568313977416
361 0.0004372491329553953
362 0.0004381708649189019
363 0.0004384264820640123
364 0.0004374491005453051
365 0.0004375510153071342
366 0.0004395974607824693
367 0.00044250949489210977
368 0.00044490387105969114
369 0.0004466667574802828
370 0.0004689649119371554
371 0.00046194544163875
372 0.00042253773664313637
373 0.00039709632922788806
374 0.00041665403732034785
375 0.0004399806587941541
376 0.0004507558158621368
377 0.00044277919408971074
378 0.000402261772909467

652 0.0002996972327420417
653 0.0002996569600064354
654 0.00029961355963169743
655 0.00029958449370485503
656 0.00029958590884570876
657 0.00029954531085062445
658 0.00029948038065182335
659 0.00029948168941657384
660 0.0002994781681614615
661 0.00029943385728447853
662 0.00029942380206656084
663 0.00029942760120594153
664 0.00029939667036409275
665 0.00029936842569131553
666 0.00029932064898967174
667 0.00029930016893408564
668 0.0002993051383807513
669 0.0002992674684169331
670 0.0002992111473793447
671 0.0002992201058573222
672 0.000299218120879673
673 0.00029918252005364484
674 0.0002991704408417861
675 0.00029918102923193944
676 0.0002991558044369028
677 0.0002991308409648085
678 0.00029908638027827594
679 0.0002990743275065647
680 0.00029908196480091486
681 0.0002990501550421896
682 0.00029899795824069503
683 0.00029900949672128773
684 0.0002990116779355633
685 0.00029897404019535987
686 0.00029896056877258206
687 0.0002989769905091533
688 0.00029895980591186254
689 0.00029894283

964 0.0003235332100046896
965 0.00031972548154215476
966 0.00032043475085040624
967 0.00032352718247421287
968 0.0003247046119222656
969 0.00032253664287225207
970 0.00031918772163520615
971 0.00031766297732740723
972 0.00031861965528936625
973 0.00032013129871374344
974 0.00032008843579826483
975 0.0003183451181773216
976 0.0003165185627885099
977 0.0003160223121157469
978 0.00031674896480888355
979 0.00031731255916283045
980 0.000316683299856869
981 0.00031519029248169004
982 0.00031411496044202734
983 0.0003140317296739278
984 0.00031443313311167433
985 0.00031442390909626716
986 0.000313745175746306
987 0.0003127846663893949
988 0.000312245151123314
989 0.00031226502426713793
990 0.0003123963422067337
991 0.0003121911098571887
992 0.0003114867201878828
993 0.000310666025741025
994 0.0003102370892526408
995 0.0003102453466902021
996 0.00031018909899291396
997 0.00030983460141359784
998 0.00030932573656918135
999 0.0003088776951282773


In [24]:
print(spam_model.init)
print(spam_model.povm)

tf.Tensor(
[[ 9.8918e-01+0.0000e+00j -8.6044e-04-1.6703e-04j  5.2985e-04+9.0636e-04j
   6.5977e-04-1.1231e-03j  2.7115e-03-2.3681e-03j  1.1205e-04+4.6617e-04j
  -5.9656e-04-1.5004e-04j -1.6698e-04-3.2826e-04j]
 [-8.6044e-04+1.6703e-04j  3.5996e-03+0.0000e+00j -3.7956e-04-2.8296e-04j
  -3.3636e-04-1.8910e-04j  4.9910e-04-7.5451e-04j  2.9802e-04+8.0696e-05j
   1.6063e-04+6.4902e-05j  2.8094e-04+3.9441e-05j]
 [ 5.2985e-04-9.0636e-04j -3.7956e-04+2.8296e-04j  5.6273e-03+0.0000e+00j
   2.2594e-04-1.3205e-04j -5.3880e-04-2.6997e-04j -8.8072e-05-7.2359e-04j
  -4.9877e-04+3.2720e-05j -2.5698e-04-1.5712e-04j]
 [ 6.5977e-04+1.1231e-03j -3.3636e-04+1.8910e-04j  2.2594e-04+1.3205e-04j
   3.0014e-04+0.0000e+00j -2.5078e-04+8.5382e-05j -5.3363e-05-1.9274e-06j
   3.8092e-05-8.7010e-05j  4.7121e-06+1.7673e-06j]
 [ 2.7115e-03+2.3681e-03j  4.9910e-04+7.5451e-04j -5.3880e-04+2.6997e-04j
  -2.5078e-04-8.5382e-05j  8.4286e-04+0.0000e+00j  1.6039e-04+1.6041e-04j
   2.6627e-05+5.1071e-05j  1.7282e-05+1.3327e